In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from cartopy import crs
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature, BORDERS

%matplotlib notebook

In [ ]:
def p25(s):
    return s.quantile(0.25)

def p75(s):
    return s.quantile(0.75)

def p90(s):
    return s.quantile(0.9)


def read_haushalte():
    df = pd.read_json('./data/haushalte.json')
    
    
    rename = {
        'Fußweg in Minuten zur nächsten Haltestelle von Bus': 'distance_bus',
        'Fußweg in Minuten zur nächsten Haltestelle von U- / Straßen- / Stadtbahn': 'distance_metro',
        'Fußweg in Minuten zur nächsten Haltestelle von S-Bahn / Nahverkehr': 'distance_train',
    }

    df.rename(columns=rename, inplace=True)
    
    numeric_columns = [
        'distance_bus',
        'distance_metro',
        'distance_train',
    ]
    
    persons = df['Anzahl der im Haushalt ständig lebenden Personen']
    
    df['cars_per_person'] = df['Anzahl der KFZ'] / persons
    df['bicycles_per_person'] = df['Anzahl der betriebsbereiten Fahrräder im HH'] / persons
    df['motorbikes_per_person'] = df['Anzahl der motorisierten Zweiräder im HH'] / persons
    
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    col = 'Wohnort Stadtbezirk Text'
    for before, after in {'ü': 'ue', 'ö': 'oe', '-': ' '}.items():
        df[col] = df[col].str.replace(before, after)
    
    return df

In [ ]:
haushalte = read_haushalte()

In [ ]:
print(*haushalte.head(), sep='\n') 

In [ ]:
haushalte['distance_bus'].hist(bins=np.arange(0, 25) - 0.5)
plt.xlabel('Fußweg zur nächsten Bushaltestelle / min')

In [ ]:
from functools import lru_cache


@lru_cache(maxsize=1)
def read_shapes():
    reader = Reader('./data/Stadtbezirk.shp')

    feature = ShapelyFeature(
        reader.geometries(),
        crs=crs.TransverseMercator(
            approx=False,
            central_longitude=9,
            false_easting=500_000,
            false_northing=0,
            scale_factor=0.9996,
            globe=crs.Globe(ellipse='GRS80'),
        ),

    )

    corrections = {
        'Schanhorst': 'Scharnhorst'
    }

    names = [
        corrections.get(r.attributes['NAME'], r.attributes['NAME'])
        for r in reader.records()
    ]
    names
    
    return reader, feature, names

In [ ]:
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from cartopy.io import img_tiles

def plot_stadtbezirke(series, label, cmap='inferno', **kwargs):
    
    reader, feature, names = read_shapes()
    
    norm = Normalize(vmin=series.min(), vmax=series.max())
    cmap = plt.get_cmap(cmap)
    sm = ScalarMappable(cmap=cmap, norm=norm)

    request = img_tiles.OSM()
   
    fig = plt.figure(constrained_layout=True, figsize=(9, 6))
    ax = fig.add_subplot(1, 1, 1, projection=request.crs)
    ax.set_extent([7.3, 7.65, 51.41, 51.61])
    ax.add_image(request, 12)
    
    values = series.loc[names] 

    artist = ax.add_feature(
        feature,
        facecolor=[cmap(norm(v)) for v in values],
        **kwargs
    )


    ax.set_title(label)
    fig.colorbar(sm, ax=ax)
    
    return ax

In [ ]:

for col in ['bus', 'metro', 'train']:

    train_distance_by_area = (
        haushalte
        .groupby('Wohnort Stadtbezirk Text')
        [f'distance_{col}']
        .agg(p75)
    )
    plot_stadtbezirke(
        train_distance_by_area,
        label=f'75% Quantil Fußweg {col} Haltestelle in Min.',
        alpha=0.5,
    )

In [ ]:
(
    haushalte
    .groupby('Wohnort Stadtbezirk Text')[['distance_bus', 'distance_metro', 'distance_train', 'Anzahl der KFZ']]
    .agg([p25, 'median', 'mean', p75])
)

In [ ]:
haushalte.groupby('Wohnort Stadtbezirk Text')['cars_per_person'].describe()

In [ ]:
plot_stadtbezirke(
    haushalte.groupby('Wohnort Stadtbezirk Text')['cars_per_person'].mean(),
    label='Mittlere Anzahl KFZs pro Person',
    alpha=0.5,
)

In [ ]:
plot_stadtbezirke(
    haushalte.groupby('Wohnort Stadtbezirk Text')['bicycles_per_person'].mean(),
    label='Mittlere Anzahl Fahrräder pro Person',
    alpha=0.5,
)

In [ ]:
plot_stadtbezirke(
    haushalte.groupby('Wohnort Stadtbezirk Text')['motorbikes_per_person'].mean(),
    label='Mittlere Anzahl Fahrräder pro Person',
    alpha=0.5,
)